In [1]:
from poisson_fem.mesh import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
mesh = PoissonFEM.RectangularMesh(np.ones(32)/32)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
mesh.setEssentialBoundary(origin)

In [5]:
# Precomputations
mesh.compGlobStiffStencil()   # very slow for large systems, but precomputed, --> no bottleneck

mesh.compSparsityPattern()

In [6]:
K0 = PETSc.Mat().createAIJ(size=(mesh.nEq, mesh.nEq), nnz=mesh.K_nnz)
lmbda = PETSc.Vec().createSeq(mesh.nCells)
Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
x = PETSc.Vec().createSeq(mesh.nEq)
rhs = PETSc.Vec().createSeq(mesh.nEq)

In [7]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [8]:
# for scipy comparison
Kes = mesh.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [9]:
N = 1e3

In [10]:
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    lmbda.setValues(range(mesh.nCells), np.ones(mesh.nCells))
    rhs.setValues(range(mesh.nEq), np.ones(mesh.nEq))
    mesh.globStiffStencil.mult(lmbda, Kvec)
    K0.setValuesCSR(mesh.K_indptr, mesh.K_indices, Kvec.getValues(mesh.Kvec_nonzero))
    K0.assemblyBegin()
    K0.assemblyEnd()

    # solving
    ksp.setOperators(K0)
    ksp.solve(rhs, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  0.00298357892036438


In [11]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ np.ones(mesh.nCells)
    K1 = sps.csr_matrix((Kvecs[mesh.Kvec_nonzero], mesh.K_indices, mesh.K_indptr))
    x1 = lg.spsolve(K1, np.ones(mesh.nEq))
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.007110557317733765


In [12]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  5.3032383964069054e-14
PETSc speedup =  2.383230847087954
